## Read-in, prepare data & load functions

In [ ]:
#pip install seaborn
#!pip install statsmodels

In [1]:
import os 
import pandas as pd
import numpy as np
import h5py
import pickle as pkl

#import functions_read_data as rdat
# Tensorflow/Keras
import tensorflow as tf
import time
print(tf.__version__)
from tensorflow import keras
from tensorflow.keras import layers
import pickle as pkl

from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

from tensorflow.keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

2.2.0


Using TensorFlow backend.


In [2]:
DIR = "/tf/notebooks/schnemau/xAI_stroke_3d/"
os.chdir(DIR)
print(os.getcwd())
IMG_DIR = "/tf/notebooks/hezo/stroke_perfusion/data/"
path_img = IMG_DIR + 'dicom_2d_192x192x3_clean_interpolated_18_02_2021_preprocessed2.h5'
path_tab = IMG_DIR + 'baseline_data_zurich_prepared.csv'

/tf/notebooks/schnemau/xAI_stroke_3d


In [3]:
import functions_metrics as fm
import functions_model_definition as md
import functions_read_data as rdat
import Utils_maurice as utils

In [4]:
# Own functions
from functions.plot_slices import plot_slices
#ontram functions
from k_ontram_functions.ontram import ontram
from k_ontram_functions.ontram_loss import ontram_loss
from k_ontram_functions.ontram_metrics import ontram_acc, ontram_auc
from k_ontram_functions.ontram_predict import predict_ontram, get_parameters
from functions.augmentation3d import zoom, rotate, flip, shift

In [5]:
which_split = 6
(X_train, X_valid, X_test, X_tab_train, X_tab_valid, X_tab_test), (Y_train, Y_valid, Y_test), results = utils.read_and_split_img_data_andrea_maurice(
    path_img = IMG_DIR + 'dicom_2d_192x192x3_clean_interpolated_18_02_2021_preprocessed2.h5', 
    path_tab = IMG_DIR + 'baseline_data_zurich_prepared.csv', 
    path_splits = '/tf/notebooks/schnemau/xAI_stroke_3d/data/andrea_splits.csv', 
    split = which_split)

Y_train_MRS = to_categorical(Y_train)
Y_valid_MRS = to_categorical(Y_valid)
Y_test_MRS = to_categorical(Y_test)

In [6]:
C = Y_train_MRS.shape[1]
batch_size = 5
num_epochs = 250
num_steps_per_epoch = 1000
ntrain = 325
nvalid = 40
ntest = 40

## Complex Intercept Image Only CI_B Ontram 

In [7]:
train_CIB_noreal = False

In [8]:
if train_CIB_noreal:
    for i in range(5):
        # call model
        input_dim = np.expand_dims(X_train, axis = -1).shape[1:]
        output_dim = C-1
        model_3d = ontram(utils.img_model_linear_final(input_dim, output_dim))         

        # Define data loaders.
        train_loader = tf.data.Dataset.from_tensor_slices((X_train[:ntrain], Y_train_MRS[:ntrain]))
        validation_loader = tf.data.Dataset.from_tensor_slices((X_valid[:nvalid], Y_valid_MRS[:nvalid]))

        def validation_preprocessing(volume, label):
            volume = tf.expand_dims(volume, axis=3)
            return volume, label

        # data augmentation 
        datagen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            zoom_range=0.15,
            shear_range=0.15,
            fill_mode="nearest")
        datagen.fit(X_train[:ntrain])

        validation_dataset = (
            validation_loader.shuffle(len(X_valid[:nvalid]))
            .map(validation_preprocessing)
            .batch(batch_size)
            .prefetch(2)
        )

        #compile
        model_3d.compile(optimizer=keras.optimizers.Adam(learning_rate=5*1e-5),
                                loss=ontram_loss(C, batch_size),
                                metrics=[ontram_acc(C, batch_size)])
        
        checkpoint_cb = keras.callbacks.ModelCheckpoint(
        filepath = os.path.join('/tf/notebooks/schnemau/xAI_stroke_3d/data/results_real/', f'CIBB_{i}.h5'),
        verbose = 1,
        save_weights_only = True,
        monitor = "val_loss", #'val_acc',
        mode = 'min',
        save_best_only = True)

        early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, restore_best_weights=True)
        
        hist = model_3d.fit(
            datagen.flow(X_train, Y_train_MRS, batch_size=batch_size, shuffle=True),
            validation_data=validation_dataset,
            epochs=num_epochs,
            shuffle=True,
            verbose=1,
            #steps_per_epoch = num_steps_per_epoch,        
            callbacks=[checkpoint_cb, early_stopping_cb])
        
        # Save the training history as a pickle file
        os.makedirs('/tf/notebooks/schnemau/xAI_stroke_3d/data/results_real/hist', exist_ok=True)
        with open(os.path.join('/tf/notebooks/schnemau/xAI_stroke_3d/data/results_real/hist', f'CIBB_{i}.pkl'), 'wb') as file:
            pkl.dump(hist.history, file, protocol=4)

## Complex Intercept Image Only CI_B Ontram different augmentation

In [9]:
train_CIB_2_noreal = True

In [ ]:
if train_CIB_2_noreal:
    for i in range(5):

        input_dim = np.expand_dims(X_train, axis = -1).shape[1:]
        output_dim = Y_test_MRS.shape[1]-1
        model_3d = ontram(utils.img_model_linear_final(input_dim, output_dim))     

        model_3d.compile(optimizer=keras.optimizers.Adam(lr = 0.001),
                                        loss=ontram_loss(C, batch_size),
                                        metrics=[ontram_acc(C, batch_size)])


        train_loader_MRS = tf.data.Dataset.from_tensor_slices((np.expand_dims(X_train, axis = -1)[:ntrain], Y_train_MRS[:ntrain]))
        validation_loader_MRS = tf.data.Dataset.from_tensor_slices((np.expand_dims(X_valid, axis = -1)[:nvalid], Y_valid_MRS[:nvalid]))
        test_loader_MRS = tf.data.Dataset.from_tensor_slices((np.expand_dims(X_test, axis = -1)[:ntest], Y_test_MRS[:ntest]))

        def train_preprocessing(data, label):
            #intercept = data[0]  # intercept
            volume = data  # shift: image
            volume = zoom(volume)
            volume = rotate(volume)
            volume = shift(volume)
            volume = flip(volume)
            return (volume), label


        train_dataset_MRS = (train_loader_MRS.shuffle(len(X_train[:ntrain]))
                            .map(train_preprocessing)
                            .batch(batch_size, drop_remainder=True))

        validation_dataset_MRS = validation_loader_MRS.batch(batch_size)             

        
        checkpoint_cb = keras.callbacks.ModelCheckpoint(
        filepath = os.path.join('/tf/notebooks/schnemau/xAI_stroke_3d/data/results_real/', f'CIB_{i}_2.h5'),
        verbose = 1,
        save_weights_only = True,
        monitor = "val_loss", #'val_acc',
        mode = 'min',
        save_best_only = True)

        early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_loss", patience=100, restore_best_weights=True)
        
        hist = model_3d.fit(
            train_dataset_MRS,
            validation_data=validation_dataset_MRS,
            epochs=250,
            shuffle=True,
            verbose=1, 
            callbacks=[checkpoint_cb, early_stopping_cb])              
        
        # Save the training history as a pickle file
        os.makedirs('/tf/notebooks/schnemau/xAI_stroke_3d/data/results_real/hist', exist_ok=True)
        with open(os.path.join('/tf/notebooks/schnemau/xAI_stroke_3d/data/results_real/hist', f'CIB_{i}_2.pkl'), 'wb') as file:
            pkl.dump(hist.history, file, protocol=4)

Epoch 1/250
65/65 [==============================] - ETA: 0s - loss: 0.5413 - acc: 0.8092
Epoch 00001: val_loss improved from inf to 0.53170, saving model to /tf/notebooks/schnemau/xAI_stroke_3d/data/results_real/CIB_0_2.h5
65/65 [==============================] - 42s 644ms/step - loss: 0.5413 - acc: 0.8092 - val_loss: 0.5317 - val_acc: 0.8250
Epoch 2/250
55/65 [========================>.....] - ETA: 6s - loss: 0.5142 - acc: 0.8036

## Complex Intercept Image CI_B and linear shift in tabular data LS_X Ontram model

In [ ]:
train_CIBLSX_noreal = False

In [ ]:
if train_CIBLSX_noreal:
    for i in range(5):
        # call model
        input_dim = np.expand_dims(X_train, axis = -1).shape[1:]
        output_dim = C-1
        CI_B = utils.img_model_linear_final(input_dim, output_dim)
        LS_X = utils.mod_linear_shift(X_tab_train.shape[1])
        model_3d = ontram(CI_B, LS_X)     

        # Define data loaders.
        train_loader = tf.data.Dataset.from_tensor_slices((X_train[:ntrain], X_tab_train[:ntrain],Y_train_MRS[:ntrain]))
        validation_loader = tf.data.Dataset.from_tensor_slices((X_valid[:nvalid], X_tab_valid[:nvalid], Y_valid_MRS[:nvalid]))

        def validation_preprocessing(volume, tabular,label):
            volume = tf.expand_dims(volume, axis=3)
            return (volume, tabular), label

        # data augmentation 
        datagen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            zoom_range=0.15,
            shear_range=0.15,
            fill_mode="nearest")
        datagen.fit(X_train[:ntrain])

        validation_dataset = (
            validation_loader.shuffle(len(X_valid[:nvalid]))
            .map(validation_preprocessing)
            .batch(batch_size)
            .prefetch(2)
        )

        #compile
        model_3d.compile(optimizer=keras.optimizers.Adam(learning_rate=5*1e-5),
                                loss=ontram_loss(C, batch_size),
                                metrics=[ontram_acc(C, batch_size)])
        
        checkpoint_cb = keras.callbacks.ModelCheckpoint(
        filepath = os.path.join('/tf/notebooks/schnemau/xAI_stroke_3d/data/results_real/', f'CIB_LSX_{i}.h5'),
        verbose = 1,
        save_weights_only = True,
        monitor = "val_loss", #'val_acc',
        mode = 'min',
        save_best_only = True)

        early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_loss", patience=100, restore_best_weights=True)
        
        hist = model_3d.fit(
            datagen.flow((X_train[:ntrain], X_tab_train[:ntrain]),Y_train_MRS[:ntrain], batch_size=batch_size, shuffle=True),
            validation_data=validation_dataset,
            epochs=250,
            shuffle=True,
            verbose=1,
            steps_per_epoch = num_steps_per_epoch,        
            callbacks=[checkpoint_cb, early_stopping_cb])
        
        # Save the training history as a pickle file
        os.makedirs('/tf/notebooks/schnemau/xAI_stroke_3d/data/results_real/hist', exist_ok=True)
        with open(os.path.join('/tf/notebooks/schnemau/xAI_stroke_3d/data/results_real/hist', f'CIB_LSX_{i}.pkl'), 'wb') as file:
            pkl.dump(hist.history, file, protocol=4)